In [ ]:
import tkinter as tk
import serial
import threading

SERIAL_PORT = 'COM5' 
BAUD_RATE = 9600

ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)

def send_to_arduino(data):
    ser.write(data.encode() + b'\n')  # Enviar cada dato con un salto de línea

def read_from_arduino():
    while True:
        if ser.in_waiting > 0:
            data = ser.readline().decode('utf-8').strip()
            display_data(data)

def display_data(data):
    oled_display.config(state='normal')
    oled_display.delete(1.0, tk.END)
    oled_display.insert(tk.END, data)
    oled_display.config(state='disabled')

current_input = ""
reset_input = False  # Variable para manejar si se debe resetear la entrada
led_on = False  # Variable para controlar el estado del LED

def key_press(key):
    global current_input, reset_input
    if reset_input:
        current_input = "" if key not in "+-*/" else current_input
        reset_input = False

    if key == '=':
        try:
            result = str(eval(current_input))
            display_data(result)
            send_to_arduino(result)  
            current_input = result
            reset_input = True
        except Exception as e:
            display_data("Error")
            send_to_arduino("Error")  # Envia un mensaje de error
            current_input = ""
            reset_input = True
    elif key == '<':
        current_input = current_input[:-1]
    else:
        current_input += key

    display_data(current_input)
    send_to_arduino(current_input) 

def check_password():
    global current_input, led_on
    password = "1245"
    if current_input == password:
        if led_on:
            display_data("Acceso Permitido - LED Apagado")
            send_to_arduino("LED_OFF")  # Comando para apagar el LED
            led_on = False  # Actualiza el estado del LED
        else:
            display_data("Acceso Permitido - LED Encendido")
            send_to_arduino("LED_ON")  # Comando para encender el LED
            led_on = True  # Actualiza el estado del LED
    else:
        display_data("Contraseña Incorrecta")
        send_to_arduino("Acceso Denegado")  # Enviar mensaje de acceso denegado
    current_input = ""

root = tk.Tk()
root.title("Interfaz Serial con Arduino")

oled_display = tk.Text(root, height=10, width=30, state='disabled')
oled_display.pack()

button_frame = tk.Frame(root)
button_frame.pack()

buttons = [
    ('1', 0, 0), ('2', 0, 1), ('3', 0, 2),
    ('4', 1, 0), ('5', 1, 1), ('6', 1, 2),
    ('7', 2, 0), ('8', 2, 1), ('9', 2, 2),
    ('0', 3, 1), ('<', 3, 0), ('=', 3, 2),
    ('+', 4, 0), ('-', 4, 1), ('*', 4, 2), ('/', 4, 3),
    ('P', 5, 0),  # Botón para comprobar la contraseña
]

for (text, row, col) in buttons:
    button = tk.Button(button_frame, text=text, command=lambda t=text: (key_press(t) if t != 'P' else check_password()), height=2, width=5)
    button.grid(row=row, column=col)

threading.Thread(target=read_from_arduino, daemon=True).start()

root.mainloop()
ser.close()